In [1]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras import layers, models


In [2]:
# Load and preprocess MNIST dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

In [15]:
X_train = X_train.reshape(-1, 28, 28, 1) / 255.0
X_test = X_test.reshape(-1, 28, 28, 1) / 255.0
# Convert labels to binary masks (simplified for demonstration)
# We treat all non-zero pixels as the digit (foreground = 1), and zeros as the background
y_train_segmentation = np.where(X_train > 0, 1, 0)
y_test_segmentation = np.where(X_test > 0, 1, 0)


In [4]:
def create_cnn_gru_model(input_shape=(28, 28, 1)):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(32, (3, 3), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(64, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Reshape((28 * 28, 64))(x)

    x = layers.GRU(128, return_sequences=True)(x)
    x = layers.GRU(64, return_sequences=True)(x)

    x = layers.Reshape((28, 28, 64))(x)

    x = layers.Conv2DTranspose(32, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    output = layers.Conv2D(1, (1, 1), activation='sigmoid')(x)

    model = models.Model(inputs, output)
    return model

In [19]:
model = create_cnn_gru_model()
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 28, 28, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 28, 28, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 28, 28, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 28, 28, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_2 (Reshape)             │ (None, 784, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, 784, 128)       │        74,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, 784, 64)        │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_3 (Reshape)             │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 28, 28, 32)     │        18,464 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 28, 28, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 28, 28, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 1)      │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 149,569 (584.25 KB)

 Trainable params: 149,313 (583.25 KB)

 Non-trainable params: 256 (1.00 KB)

In [21]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [23]:
EPOCHS = 10
BATCH_SIZE = 32


In [ ]:
history = model.fit(X_train, y_train_segmentation, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(X_test, y_test_segmentation),steps_per_epoch=10)

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 273s 27s/step - accuracy: 0.7374 - loss: 0.5195 - val_accuracy: 0.5001 - val_loss: 0.6934
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 289s 32s/step - accuracy: 0.9261 - loss: 0.2328 - val_accuracy: 0.5001 - val_loss: 0.7105
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9364 - loss: 0.1969

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test_bin)
print(f"Test Accuracy: {test_acc}")

In [ ]:
predictions = model.predict(x_test)
predictions

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def visualize_results(images, masks, preds, num_examples=5):
    for i in range(num_examples):
        plt.figure(figsize=(10, 3))
        # Original image
        plt.subplot(1, 3, 1)
        plt.title('Original Image')
        plt.imshow(images[i].reshape(28, 28), cmap='gray')

        # Ground truth segmentation mask
        plt.subplot(1, 3, 2)
        plt.title('Ground Truth Mask')
        plt.imshow(masks[i].reshape(28, 28), cmap='gray')

        # Predicted segmentation mask
        plt.subplot(1, 3, 3)
        plt.title('Predicted Mask')
        plt.imshow(preds[i].reshape(28, 28), cmap='gray')
        plt.show()


In [ ]:
visualize_results(x_test, y_test_segmentation, predictions)
